In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
resnet50 = models.resnet50(pretrained=True)
modules=list(resnet50.children())[:-1]
resnet50=nn.Sequential(*modules)
for p in resnet50.parameters():
    p.requires_grad = False

In [3]:
import PIL
import numpy as np
from torchvision.transforms import transforms

In [4]:
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])

In [7]:
def encode(img_path):
    img = PIL.Image.open(img_path)
    image_tensor = test_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    img_var = Variable(image_tensor) # assign it to a variable
    features_var = resnet50(img_var) # get the output from the last hidden layer of the pretrained resnet
    features = features_var.data # get the tensor out of the variable
    return features.flatten()

In [5]:
df = spark.read.option("header",True).csv("./data_10.csv")

In [6]:
def f(x):
    x["vector"] = 1

In [7]:
df.foreach(lambda x: f(x["path"]))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (172.31.93.176 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hieu/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/hieu/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/home/hieu/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/hieu/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/hi

In [53]:
dictionary

{}

In [ ]:
import os

pathList = []

idx = 0


for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        if (idx < 5):
            
            result = result.tolist()
            result.insert(0, os.path.join(dirname, filename))
            print(result)
            dataList.append(result)
            #print(result.tolist())
            idx += 1

print(dataList)
columns = [str(i) for i in range(1, 2049, 1)]
columns.insert(0, "path")
print(len(columns))
df = spark.createDataFrame(dataList, columns)
  
df.show()

['./data/0198714006.jpg', 0.40350058674812317, 0.5392924547195435, 0.4854702055454254, 0.3812853693962097, 0.537124752998352, 0.49192845821380615, 0.5121925473213196, 0.49884405732154846, 0.42239344120025635, 0.41514840722084045, 0.29119470715522766, 0.38561758399009705, 0.549205482006073, 0.5208191275596619, 0.44188645482063293, 0.5447995066642761, 0.5603684186935425, 0.2667138874530792, 0.3801047205924988, 0.36360326409339905, 0.43657422065734863, 0.4703162908554077, 0.41399693489074707, 0.3927305340766907, 0.572419285774231, 0.5486962199211121, 0.45506158471107483, 0.43639254570007324, 0.43920305371284485, 0.4061703681945801, 0.4106246531009674, 0.41734859347343445, 0.4409765899181366, 0.4444539546966553, 0.6827478408813477, 0.5146545171737671, 0.47448012232780457, 0.47115737199783325, 0.42158597707748413, 0.5273796916007996, 0.45079734921455383, 0.5016883015632629, 0.3640803396701813, 0.36551952362060547, 0.3457994759082794, 0.49978920817375183, 0.42854249477386475, 0.6047426462173

In [ ]:
#test = encode("0120129014.jpg")
#test

In [19]:
!unzip data.zip -d image

Archive:  data.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data.zip or
        data.zip.zip, and cannot find data.zip.ZIP, period.


In [20]:
!ls 

data_10.csv  data123.zip  data.zip  sample_data


In [2]:
import os
import pandas as pd

In [3]:
list_file = os.listdir("./data")

In [6]:
df = pd.DataFrame(list_file, columns={"path"})

In [8]:
df['path'] = "./data/" + df['path']

In [10]:
df.to_csv("data_10.csv", index = False)

In [27]:
list_string  = [0.123, 0.456, 0.789]

In [28]:
list_string

[0.123, 0.456, 0.789]

In [23]:
listToStr = ','.join([str(elem) for elem in list_string])

In [25]:
listToStr

'0.123,0.456,0.789'

In [17]:
listToStr.split(",")

['0.123', '0.456', '0.789']

In [18]:
listToStr

'0.123,0.456,0.789'